In [ ]:
import numpy as np
import pandas as pd
import os
import librosa
from IPython import display as ipd
import soundfile as sf
from tqdm import tqdm
from libsoni.core.pianoroll import sonify_pianoroll_additive_synthesis, \
                                   sonify_pianoroll_fm_synthesis, \
                                   sonify_pianoroll_clicks, \
                                   sonify_pianoroll_sample, \
                                   visualize_pianoroll

from libsoni.core.tse import sonify_tse_clicks
from libsoni.util.utils import mix_sonification_and_original, get_preset

ANNOTATION_DIR = '/home/yiitozer/workspace/python/synctoolbox/PCD_MuseScore/PCD_sync/'
PCD_DIR = '/run/user/1001/gvfs/smb-share:server=lin2.audiolabs.uni-erlangen.de,share=groupmm/AL_Projects/PianoConcerto_MMO/MMO_rendered/PCD/excerpts'

Fs = 22050
PC_IDS = sorted(os.listdir(ANNOTATION_DIR))

In [ ]:
instruments = 'O'
for pc_id in tqdm(PC_IDS):
    try:
        pc_df = pd.read_csv(os.path.join(ANNOTATION_DIR, pc_id, pc_id + f'_{instruments}_sync.csv'), sep=';')
    except:
        continue
    if os.path.isfile(os.path.join(ANNOTATION_DIR, pc_id, pc_id + f'_{instruments}_soni.wav')):
        continue
    
    else:
        print(pc_id)
    
    pc_audio, _ = librosa.load(os.path.join(PCD_DIR, pc_id, pc_id + f'_{instruments}.wav'), sr=Fs)
    fm = sonify_pianoroll_additive_synthesis(pianoroll_df=pc_df,
                                             partials=[1, 2, 3],
                                             partials_amplitudes=[0.5, 0.25, 0.25])
    clicks = sonify_pianoroll_clicks(pianoroll_df=pc_df)
    
    soni = fm + clicks * 10

    sonification_stereo = mix_sonification_and_original(soni,pc_audio, gain_lin_sonification=10)
    sonification_stereo /= sonification_stereo.max()
    
    #son = np.zeros(sonification_stereo.shape)
    #son[0, :] = sonification_stereo[1, :]
    #son[1, :] = sonification_stereo[0, :]

    sf.write(os.path.join(ANNOTATION_DIR, pc_id, pc_id + f'_{instruments}_soni.wav'), sonification_stereo.T, Fs)